# [<span role="heading" aria-level="2" style="margin: 0px; padding: 0px; vertical-align: middle;">按多个关键字查询（sql）</span>](https:\www.cnblogs.com\longdexinoy\archive\2012\09\07\2676022.html)

### 1\. 标量值函数 fn\_GetMoreKeyWordsSelectExpression

In [ ]:
CREATE FUNCTION [dbo].[fn_GetMoreKeyWordsSelectExpression] 
(
    -- Add the parameters for the function here
    @KeyWords nvarchar(500), --@KeyWords 指：关键字字符串，如 'voa,vs,kk,shouji'
    @Operation nvarchar(50),    --@Operation 指：AND 或 OR 关键字
    @ColumName nvarchar(100) --@ColumName 指：列名
)
RETURNS nvarchar(4000)
AS
BEGIN
    -- Declare the return variable here
    declare @position int
    declare @KeyWord nvarchar(500)
    declare @Expression nvarchar(4000)

--    set @ColumName = 'Topics'
--    set @KeyWords = 'voa,vs,kk'
--    set @Expression = ''
--    set @Operation = ' or '

    set @Expression = ''
    while (len(@KeyWords) > 0)
    begin
        set @position = charindex(',',@KeyWords)
        if @position > 0
        begin
            set @KeyWord = substring(@KeyWords,1,@position - 1)
            set @Expression = @Expression + ' ' + @Operation  + ' ' + @ColumName + ' like ' + '''' + '%' + @KeyWord + '%' + ''''
            set @KeyWords = substring(@KeyWords,@position + 1,len(@KeyWords))
        end
        else
        begin
            set @Expression = @Expression + ' ' + @Operation + ' ' + @ColumName + ' like ' + '''' + '%' + @KeyWords + '%' + ''''
            set @KeyWords = ''
        end
    end

    set @Expression = substring(@Expression,charindex(@Operation,@Expression) + 3,len(@Expression))

    -- Return the result of the function
    RETURN (@Expression)

END

### <span style="color: rgb(17, 17, 17); font-family: &quot;PingFang SC&quot;, &quot;Microsoft YaHei&quot;, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 13px; background-color: rgb(255, 255, 255);">2.存储过程 up_SelectByMoreKeyWords</span>

In [ ]:
CREATE PROCEDURE [dbo].[up_SelectByMoreKeyWords]
    @Flag nvarchar(100), 
    @KeyWords nvarchar(500), --@KeyWords 指：关键字字符串，如 'voa,vs,kk,shouji'
    @SearchBy nvarchar(100) --@SearchBy 指：按具体列明来查询，如 Topics，Contents
AS
BEGIN
    -- SET NOCOUNT ON added to prevent extra result sets from
    -- interfering with SELECT statements.
    SET NOCOUNT ON;
    
    declare @sql nvarchar(4000)
    declare @sql1 nvarchar(4000)
    declare @sql2 nvarchar(4000)

    set @sql1 = 'select 
    t.TopicsId,
    t.Topics,
    p.Contents,
    t.UserName,
    t.CreateTime
    from dbo.BBS_Topics t
    join dbo.BBS_Posts p  on t.TopicsId = p.TopicsId
    where p.LayId = 0 and t.DisplayModes > 0 and (' + dbo.fn_GetMoreKeyWordsSelectExpression(@KeyWords,'or','p.' + @SearchBy) + ')'

    set @sql2 = 'select 
    t.TopicsId,
    t.Topics,
    p.Contents,
    t.UserName,
    t.CreateTime
    from dbo.BBS_Topics t
    join dbo.BBS_Posts p  on t.TopicsId = p.TopicsId
    where p.LayId = 0 and t.DisplayModes > 0 and (' + dbo.fn_GetMoreKeyWordsSelectExpression(@KeyWords,'and','p.' + @SearchBy) + ')'


    if @flag = 'or'
    set @sql = @sql1
    else
    set @sql = @sql2

    --print @sql
    exec (@sql)

END